In [21]:
import googlemaps
import numpy as np
import pandas as pd
from matplotlib import font_manager, rc
import matplotlib.pyplot as plt
import platform
import seaborn as sns
from sklearn import preprocessing
import folium

In [22]:
plt.rcParams['axes.unicode_minus'] = False
if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system... sorry~~~~')

In [23]:
df = pd.read_excel("SeoulBiman.xlsx", header=2, encoding='utf-8')
df.drop([0], inplace=True)
df.head()

,기간,자치구,비만(BMI≥25),저체중(BMI<18.5),정상(18.5≤BMI＜25),체질량지수(BMI)(평균)
1,2018,종로구,34.5,5.7,59.8,23.8
2,2018,중구,31.6,8.8,59.6,23.2
3,2018,용산구,30.2,1.1,68.7,23.8
4,2018,성동구,32.8,4.3,63,23.7
5,2018,광진구,30.5,6.2,63.3,23.5


In [24]:
df.rename(columns={'비만(BMI≥25)':'비만', '저체중(BMI<18.5)':'저체중', 
                   '정상(18.5≤BMI＜25)':'정상','체질량지수(BMI)(평균)':'BMI'}, inplace=True)
df.head(2)

,기간,자치구,비만,저체중,정상,BMI
1,2018,종로구,34.5,5.7,59.8,23.8
2,2018,중구,31.6,8.8,59.6,23.2


In [25]:
col=['비만','저체중','정상']

x=df[col].values
min_max_scalr=preprocessing.MinMaxScaler()

x_scaled=min_max_scalr.fit_transform(x.astype(float))
df_norm = pd.DataFrame(x_scaled, columns=col, index=df.index)

col2 = ['기간','자치구','BMI']
df_norm[col2]=df[col2]
df_norm.head()

,비만,저체중,정상,기간,자치구,BMI
1,0.735294,0.597403,0.282759,2018,종로구,23.8
2,0.564706,1.000000,0.268966,2018,중구,23.2
3,0.482353,0.000000,0.896552,2018,용산구,23.8
4,0.635294,0.415584,0.503448,2018,성동구,23.7
5,0.500000,0.662338,0.524138,2018,광진구,23.5


In [26]:
df_norm.set_index('자치구', inplace=True)
df_norm.head()

,비만,저체중,정상,기간,BMI
자치구,,,,,
종로구,0.735294,0.597403,0.282759,2018,23.8
중구,0.564706,1.000000,0.268966,2018,23.2
용산구,0.482353,0.000000,0.896552,2018,23.8
성동구,0.635294,0.415584,0.503448,2018,23.7
광진구,0.500000,0.662338,0.524138,2018,23.5


In [27]:
import json
geo_path='02. skorea_municipalities_geo_simple.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))

In [28]:
map = folium.Map(location=[37.5502, 126.982], zoom_start=11, tiles='Stamen Toner')
map.choropleth(geo_data=geo_str, data=df_norm['BMI'], columns = [df_norm.index, df_norm['BMI']],
               fill_color='PuRd', key_on='feature.id')
map